# 1. Leer datos

# Funcion para leer el set de datos 


In [4]:
import numpy as np 
import os
import gzip

def cargar_set(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' %tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' %tipo)

  with gzip.open(ruta_categorias, 'rb') as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype = np.uint8, offset = 8 )
  
  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype = np.uint8, offset = 16).reshape(len(etiquetas),784)

  return imagenes, etiquetas

# obtener acceso a el set de datos en drive


In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ruta = 'gdrive/My Drive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = cargar_set(ruta, tipo='train')
X_test, Y_test = cargar_set(ruta, tipo='test')


Mounted at /content/gdrive


nota: para este caso me presento un error con la ruta del la carpeta mnist, al entender de manera autonoma que el google colab crea una carpeta en el drive de tu cuenta actual, procedi a agregar la carpeta fashion_mnist descargada del repositorio y al ejecutar de nuevo ya estaba arreglado.


# Reshape de los datos para garantizar que sean matrices

In [7]:
X_train = X_train[0:59904,:]
X_test = X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train, (59904,28,28,1))
X_test =np.reshape(X_test, (9984,28,28,1))

# Importando Tensorflow 2

In [8]:
%tensorflow_version 2.x
import tensorflow as tf

print('Version de Tensorflow: ' + tf.__version__)



Version de Tensorflow: 2.4.1


# Creacion del modelo


In [11]:
tf.random.set_seed(200)

modelo = tf.keras.models.Sequential()

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(64,(5,5), padding='same', activation='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(128,(5,5), padding='same', activation='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(256,(5,5), padding='same', activation='elu'))
modelo.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.Flatten())
# red neuronal de 256 neuronas
modelo.add(tf.keras.layers.Dense(256))
modelo.add(tf.keras.layers.Activation('elu'))
modelo.add(tf.keras.layers.Dropout(0.5))
modelo.add(tf.keras.layers.Dense(10))
modelo.add(tf.keras.layers.Activation('softmax'))

#imprimir informacion del modelo
modelo.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_6 (Batch (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 128)        

# Entrenamiento 1 con CPU

In [12]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [13]:
import timeit 

def entrenamiento_cpu():
  with tf.device('/cpu:0'):
    modelo.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128,epochs=2,verbose=1)
  return None

cpu_time = timeit.timeit('entrenamiento_cpu()', number=1, setup='from __main__ import entrenamiento_cpu')


Epoch 1/2
468/468 [==============================] - 753s 2s/step - loss: 1.0169 - accuracy: 0.7078 - val_loss: 0.4264 - val_accuracy: 0.8551
Epoch 2/2
468/468 [==============================] - 757s 2s/step - loss: 0.4105 - accuracy: 0.8562 - val_loss: 0.3127 - val_accuracy: 0.8922


In [14]:
print('Tiempo de entrenamiento ' + str(cpu_time) + 'segundos ')

Tiempo de entrenamiento 1509.695826668segundos 
